In [21]:
import os
import json
import pandas as pd
if os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [22]:
from main import run
from utils.gcp_helpers import get_dataframe_from_bucket

In [23]:
# create train data
if False:
    data = get_dataframe_from_bucket("Radiation/mouse/combined/mouse.combined.expression.071121.tsv","ah21_data", "ah21_data", sep="\t")
    data = data.T
    data.columns = data.iloc[0]
    data = data.iloc[1:]
    data = data.astype(float)
    metadata = get_dataframe_from_bucket("Radiation/mouse/combined/mouse.metadata.combined.071121.csv","ah21_data", "ah21_data", sep=",")
    merged_data = pd.merge(metadata, data, left_on="sample", right_index=True)
    merged_data = merged_data[merged_data.dataset == "GSE124612"]
    merged_data["irradiated"] = [1. if x == "Yes" else 0. for x in  merged_data.irradiated.values]
    merged_data.to_pickle(dataset_file)

In [24]:
dataset_file = "data/mouse_crisp_input.pkl"
verbose = 1
output_data_regime = 'binary' #'real-valued'

In [25]:
merged_data_columns = pd.read_pickle('data/mouse_crisp_input.pkl').columns

In [26]:
merged_data_values = pd.read_pickle('data/mouse_crisp_input.pkl')[merged_data_columns[9:]]#.astype(float) 

In [27]:
#correl = merged_data_values.corr()
# correl.shape

In [28]:
# define config
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": verbose,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        #'dataset_fp': '../data/test_multiclass.pkl',
        'synthetic_train_test_split' : False,
        #'dataset_fp' : outfile,
        #'dataset_fp_train' : outfile_train,
#         'dataset_fp_test' : None,
        'dataset_fp' : dataset_file,
        'output_data_regime' : output_data_regime,
        'subject_keys': 'sample',
        'targets': ['irradiated'], #['dose_Gy'],
        #'predictors': ['All'],
        'predictors': list(merged_data_columns[9:]),
        'environments': ['timepoint_hr'],
        'exclude': ['organism', 'gender', 'strain', 'radiation', 'dose_Gy']
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": verbose,
        "seed": 12,
        "variance_threshold" : 0.25, # actually std
    },
    "ensemble_options": {
        "models": ["LIRM", "ICP",], # "DCF", "LIRM", "NLICP"],
        "LIRM" : {
            "use_icp_initialization": False,
            "verbose": verbose,
            "n_iterations": 1000,
            "seed": 0,
            "lr": 0.001,
            "cuda": False,
            "output_data_regime" : output_data_regime,
        },
        "DCF" : {
            "minP": 0.1,
            "maxP": 0.9,
            "minFeatures": 1,
            "minAccuracy": 0.5,
            "seed": 0,
            "verbose": verbose,
            "target": ["Target"],
            "output_pvals": False,
            "output_data_regime" : output_data_regime,
        },
        "NLICP" : {
            "max_set_size": 2,
            "alpha": 0.05,
            "seed": 12,
            "verbose": verbose,
            "method": "MLP",
            "hidden_dim": 256,
            "output_data_regime" : output_data_regime,
        },
        "LICP" : {
            "max_set_size": 2,
            "alpha": 0.05,
            "seed": 12,
            "verbose": verbose,
            "output_data_regime" : output_data_regime,
        },
    },
     "use_cloud":False,
    "results_directory": "results/"
}

In [29]:
# Reinitialise datasets without zero variance columns
from dataio.datasets import get_datasets_for_experiment
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

######################### LIST CHOSEN ENSEMBLE METHODS #############################
ensemble_options = config.get('ensemble_options', {})
data_config = config.get('data_options', {})
selection_config = config.get('feature_selection_options', {})
# Load list of models to include in ensemble, if not specified, run them all
selected_models = ensemble_options.get("models", ["ERM", "RF", "ICP", "IRM", "DCF", "LIRM", "NLICP"])
print("Running Ensemble with the following models: ", selected_models)
# Initialise empty list to store per method results to outputs to file system/ cloud storage
to_bucket_results = []

############################# FEATURE REDUCTION 1 ######################################
# 1. Remove any zero variance features
# - Save list to results directory
from utils.ZeroVarianceChecker import ZeroVarianceChecker
print('Checking for features with zero variance')
# Flag for checking zero variance across all environments ([train_environments], val, test) or within any one environment
var_args = {
    'in_each_env': selection_config.get('zero_variance_in_each_env', False),
    'variance_threshold': selection_config.get('variance_threshold',0)
}
zero_var_checker = ZeroVarianceChecker(environment_datasets, val_dataset, test_dataset, var_args)
if var_args['in_each_env']:
    zero_var_columns = zero_var_checker.zero_var_cols
    if config['verbose']:
        print('In atleast one environment the following columns had zero variance:', zero_var_columns)
    save_dict_to_json({'zero std columns removed': zero_var_columns},
                      config['results_directory'] + 'zero_std_columns.json')
else:
    zero_var_columns = zero_var_checker.zero_var_cols
    if config['verbose']:
        print('Across all environments the following columns had zero variance:', zero_var_columns)
keep_columns = zero_var_checker.reduced_feature_list()

selected_feature_list = keep_columns
config['data_options']['predictors'] = selected_feature_list
# Reinitialise datasets without zero variance columns
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

Running a per sample experiment
Loaded  6  train environments
Env  0  has  11  samples
X shape  (11, 16353)  y shape  (11, 1)
Env  1  has  82  samples
X shape  (82, 16353)  y shape  (82, 1)
Env  2  has  40  samples
X shape  (40, 16353)  y shape  (40, 1)
Env  3  has  41  samples
X shape  (41, 16353)  y shape  (41, 1)
Env  4  has  38  samples
X shape  (38, 16353)  y shape  (38, 1)
Env  5  has  43  samples
X shape  (43, 16353)  y shape  (43, 1)
Loaded test set, X shape: (31, 16353)  y shape:  (31, 1)
Running Ensemble with the following models:  ['LIRM', 'ICP']
Checking for features with zero variance
variance threshold is 0.25
Across all environments the following columns had zero variance: ['0610005C13Rik', '0610007C21Rik', '0610007L01Rik', '0610007P08Rik', '0610007P14Rik', '0610007P22Rik', '0610008F07Rik', '0610009B22Rik', '0610009D07Rik', '0610009K14Rik', '0610009O20Rik', '0610010F05Rik', '0610010K14Rik', '0610010O12Rik', '0610011L14Rik', '0610012G03Rik', '0610012H03Rik', '0610030E20Ri

In [10]:
# Reinitialise datasets without zero variance columns
#from dataio.datasets import get_datasets_for_experiment
#environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

######################### LIST CHOSEN ENSEMBLE METHODS #############################
##ensemble_options = config.get('ensemble_options', {})
#data_config = config.get('data_options', {})
#selection_config = config.get('feature_selection_options', {})
# Load list of models to include in ensemble, if not specified, run them all
#selected_models = ensemble_options.get("models", ["ERM", "RF", "ICP", "IRM", "DCF", "LIRM", "NLICP"])
#print("Running Ensemble with the following models: ", selected_models)
# Initialise empty list to store per method results to outputs to file system/ cloud storage
#to_bucket_results = []

# 2. Correlation Analysis
# - for pairs of features with high correlation, remove one of each pair
from models.Investigate_Correlation import Correlation
print('Running correlation analysis')
corr_args = {
    "correlation_threshold": selection_config.get("correlation_threshold", 0.97),
    "seed": selection_config.get("seed", 12),
    "verbose": selection_config.get("verbose", 0),
    "columns": test_dataset.predictor_columns,
    "target": data_config['targets'],
    "save_plot": selection_config.get('plot_correlation', False),
    "fname": config['results_directory']
}

Running correlation analysis


In [11]:
corr = Correlation(environment_datasets, val_dataset, test_dataset, corr_args)

Intersection: [('Cbs', 'Rarres2'), ('Hpd', 'Reep6'), ('Idua', 'Slc17a5'), ('Rarres2', 'Serpinc1'), ('Hpn', 'Pipox'), ('Aadac', 'Acox2'), ('Cfi', 'Mosc1'), ('C030007I01Rik', 'Pdzrn3'), ('C8g', 'Mug1'), ('Afm', 'Rdh7'), ('Apom', 'Serpinf2'), ('E1A_r60_a97', 'E1A_r60_n11'), ('Ddt', 'Hint2'), ('Acadvl', 'Ebp'), ('Ddt', 'Gcgr'), ('C1qa', 'C1qc'), ('1190003J15Rik', 'Cbs'), ('C4bp', 'Cfi'), ('Apoc3', 'Serpina3k'), ('Gstz1', 'Proz'), ('C8g', 'Serpinf2'), ('Lgals4', 'Tff3'), ('Ass1', 'Gnmt'), ('5330437I02Rik', 'Pdzrn3'), ('Col18a1', 'Cyp2d10'), ('Coasy', 'Ftcd'), ('Atp5g3', 'Ndufs5'), ('Aldh7a1', 'Cyp2a12'), ('Cyp2a12', 'Slc27a5'), ('Asgr2', 'Coasy'), ('Fgf23', 'Slc17a5'), ('F2', 'Serpinc1'), ('Cfi', 'Rdh7'), ('C8g', 'Rarres2'), ('Abhd14b', 'Ddt'), ('Mogat2', 'Slc6a8'), ('Got2', 'Trfr2'), ('Gcat', 'Pecr'), ('Azgp1', 'C4bp'), ('Ddt', 'Hal'), ('1190003J15Rik', 'Serpina3m'), ('Gc', 'Serpinf2'), ('1190003J15Rik', 'Asgr2'), ('Gnmt', 'Ttr'), ('Reep6', 'Tmprss6'), ('Cgref1', 'Mogat2'), ('Apoh', 'Azgp1

In [12]:
corr_results_dict = corr.predictor_results()

In [13]:
keep_columns = corr_results_dict["results"]['retained_columns']
column_pairs_df = corr_results_dict["results"]['invariant_correlations']

selected_feature_list = keep_columns
original_dimensionality = len(selected_feature_list)
config['data_options']['predictors'] = selected_feature_list
# Reinitialise datasets without highly correlated feature pairs
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

Running a per sample experiment
Loaded  5  train environments
Env  0  has  41  samples
X shape  (41, 1148)  y shape  (41, 1)
Env  1  has  42  samples
X shape  (42, 1148)  y shape  (42, 1)
Env  2  has  39  samples
X shape  (39, 1148)  y shape  (39, 1)
Env  3  has  41  samples
X shape  (41, 1148)  y shape  (41, 1)
Env  4  has  39  samples
X shape  (39, 1148)  y shape  (39, 1)
Loaded val set, X shape: (22, 1148)  y shape:  (22, 1)
Loaded test set, X shape: (24, 1148)  y shape:  (24, 1)


In [ ]:
run(config)

Running a per sample experiment
Loaded  6  train environments
Env  0  has  15  samples
X shape  (15, 1412)  y shape  (15, 1)
Env  1  has  82  samples
X shape  (82, 1412)  y shape  (82, 1)
Env  2  has  43  samples
X shape  (43, 1412)  y shape  (43, 1)
Env  3  has  35  samples
X shape  (35, 1412)  y shape  (35, 1)
Env  4  has  40  samples
X shape  (40, 1412)  y shape  (40, 1)
Env  5  has  40  samples
X shape  (40, 1412)  y shape  (40, 1)
Loaded val set, X shape: (28, 1412)  y shape:  (28, 1)
Loaded test set, X shape: (31, 1412)  y shape:  (31, 1)
Running Ensemble with the following models:  ['LIRM', 'ICP']


/home/jupyter/crisp/utils/CorrelationToTarget.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  all_x = np.array(all_x)
/home/jupyter/crisp/utils/CorrelationToTarget.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  all_y = np.array(all_y)


Checking for features with zero variance
variance threshold is 0.25
Across all environments the following columns had zero variance: []
Running a per sample experiment
Loaded  6  train environments
Env  0  has  15  samples
X shape  (15, 1412)  y shape  (15, 1)
Env  1  has  82  samples
X shape  (82, 1412)  y shape  (82, 1)
Env  2  has  43  samples
X shape  (43, 1412)  y shape  (43, 1)
Env  3  has  35  samples
X shape  (35, 1412)  y shape  (35, 1)
Env  4  has  40  samples
X shape  (40, 1412)  y shape  (40, 1)
Env  5  has  40  samples
X shape  (40, 1412)  y shape  (40, 1)
Loaded val set, X shape: (28, 1412)  y shape:  (28, 1)
Loaded test set, X shape: (31, 1412)  y shape:  (31, 1)
Running correlation analysis
Intersection: [('Apoc4', 'Fbp1'), ('C8g', 'Fbp1'), ('Ahsg', 'Serpina1e'), ('Pou3f1', 'Rgs7'), ('Col18a1', 'Trfr2'), ('Pon1', 'Serpind1'), ('Creb3l3', 'Ephx2'), ('Gnmt', 'Proc'), ('Cgref1', 'Slc6a8'), ('Col18a1', 'Serpinf2'), ('Pzp', 'Serpinc1'), ('Fbp1', 'Slc27a5'), ('4930523O13Rik',

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


iteration: 200 error: 83.79419708251953
iteration: 400 error: 62.083309173583984
iteration: 600 error: 39.17180252075195
iteration: 800 error: 35.073848724365234
IRM (reg=0.000) has 8.133 validation error.
	new best model:
	error: tensor(8.1327)
	reg: 0
	phi.trace: tensor(1297.0992, grad_fn=<TraceBackward>)
iteration: 0 error: 744.7650146484375
iteration: 200 error: 150.5603485107422
iteration: 400 error: 76.18623352050781
iteration: 600 error: 54.21657180786133
iteration: 800 error: 35.1220703125
IRM (reg=0.000) has 24.606 validation error.
iteration: 0 error: 744.7650146484375
iteration: 200 error: 12.765205383300781
iteration: 400 error: 9.553281784057617
